# 그래디언트 클리핑(Gradient Clipping)

그래디언트 클리핑(Gradient Clipping)은 그래디언트 폭주(exploding gradient) 문제를 줄이는 방법이며, 역전파(backprop) 단계에서 그래디언트 값이 아래의 그림과 같이 특정 임계값(threshold)을 넘지 않도록 잘라내는 방법입니다.

![그래디언트 클리핑](https://github.com/GDGoC-SCHU/2024-ai-study/blob/main/week5/task1/png/Gradient%20Clipping.png?raw=true)

## 텐서플로에서 그래디언트 클리핑 구현하기

텐서플로에서는 `tf.clip_by_value`를 이용해 그래디언트 클리핑을 구현할 수 있습니다. 아래의 예제는 MNIST 데이터셋을 분류하는 간단한 분류기를 구현한 뒤에 그래디언트 클리핑을 적용한 예제입니다. 아래의 전체 코드는 [ExcelsiorCJH](https://github.com/ExcelsiorCJH/Hands-On-ML/blob/master/Chap11-Training_DNN/Chap11_2-Training_DNN.ipynb) GitHub에서 확인할 수 있습니다. `tf.clip_by_value`를 사용하려면, 아래의 코드에서 옵티마이저(`tf.train.GradientDescentOptimizer`)에 사용해야 합니다.

In [ ]:
reset_graph()

################
# layer params #
################
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

# input layer
inputs = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
# output layer
labels = tf.placeholder(tf.int32, shape=[None], name='labels')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(inputs, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name='hidden2')
    logits = tf.layers.dense(hidden2, n_outputs, name='logits')

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

################
# Hyper-params #
################
learning_rate = 0.01
threshold = 1.0
n_epochs = 5
batch_size = 50

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # 그래디언트 계산
    grad_and_vars = optimizer.compute_gradients(loss)
    # 그래디언트 클리핑
    clipped_grads = [(tf.clip_by_value(grad, -threshold, threshold), var)
                     for grad, var in grad_and_vars]
    # 클리핑 된 그래디언트 적용
    train_op = optimizer.apply_gradients(clipped_grads)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(train_op, feed_dict={inputs: batch_x,
                                          labels:batch_y})

        # validation
        accuracy_val = accuracy.eval(feed_dict={inputs: valid_x, labels: valid_y})
        print('epoch: {:03d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

epoch: 000, valid. Acc: 0.9026
epoch: 001, valid. Acc: 0.9244
epoch: 002, valid. Acc: 0.9362
epoch: 003, valid. Acc: 0.9410
epoch: 004, valid. Acc: 0.9460


# 심층 신경망 학습 - 학습된 모델 재사용하기

## Set Up

In [ ]:
import os
import numpy as np
import tensorflow as tf

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
# matplotlib.rc('font', family='AppleGothic')  # MacOS
matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

규모가 매우 큰 DNN 모델을 학습 시킬 때 처음부터 새로 학습 시키는 것은 학습 속도가 느린 문제가 있습니다. 이러한 경우 기존에 학습된 비슷한 DNN모델이 있을 때 이 모델의 하위층(lower layer)을 가져와 재사용하는 것이 학습 속도를 빠르게 할 수 있을 뿐만아니라 학습에 필요한 Training set도 훨씬 적습니다.

예를 들어, 아래의 그림처럼 [CIFAR10](http://www.cs.toronto.edu/~kriz/cifar.html) 데이터셋을 분류(비행기, 자동차, 새, 고양이, 사슴, 개, 개구리, 말, 배, 트럭의 10개  클래스)하는 모델 A가 있고, 분류된 CIFAR10 이미지에서 자동차의 종류를 분류하는 모델인 B를 학습시킨다고 할 때 학습된 모델 A에서의 일부분(lower layer)을 재사용하여 모델 B를 학습 시킬 수 있습니다. 이러한 방법을 **Transfer Learning**이라고 합니다.

![Transfer Learning](https://github.com/GDGoC-SCHU/2024-ai-study/blob/main/week5/task1/png/Transfer%20Learning.png?raw=true)


## 텐서플로 모델 재사용하기

텐서플로에서는 사전에 학습된 모델을 복원하여 새로운 모델을 학습시키는 데 사용할 수 있습니다. 텐서플로의 `tf.train.Saver`클래스를 이용해 학습된 모델을 저장하고 복원할 수 있게합니다.

### 학습된 모델 저장하기

아래의 예제 코드는, 5개의 hidden layer로 구성된 MNIST 데이터셋을 분류하는 모델입니다. `tf.train.Saver`를 이용해 학습된 모델을 `'my_model.ckpt'`에 저장하는 코드입니다.

In [ ]:
import os
import numpy as np
import tensorflow as tf

# MNIST Load
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

# Train & TestSet reshape
train_x = train_x.astype(np.float32).reshape(-1, 28*28) / 255.
train_y = train_y.astype(np.int32)
test_x = test_x.astype(np.float32).reshape(-1, 28*28) / 255.
test_y = test_y.astype(np.int32)

# Split Validation set from Train set
valid_x, train_x = train_x[:5000], train_x[5000:]
valid_y, train_y = train_y[:5000], train_y[5000:]

In [ ]:
def shuffle_batch(inputs, labels, batch_size):
    rnd_idx = np.random.permutation(len(inputs))
    n_batches = len(inputs) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_x, batch_y = inputs[batch_idx], labels[batch_idx]
        yield batch_x, batch_y

In [ ]:
reset_graph()

################
# layer params #
################
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

# input layer
inputs = tf.placeholder(tf.float32, shape=[None, n_inputs], name="inputs")
# output layer
labels = tf.placeholder(tf.int32, shape=[None], name='labels')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(inputs, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name='hidden2')
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name='hidden3')
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name='hidden4')
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name='hidden5')
    logits = tf.layers.dense(hidden5, n_outputs, name='logits')

with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

################
# Hyper-params #
################
learning_rate = 0.01
n_epochs = 5
batch_size = 50

with tf.name_scope('train'):
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# Saver 정의
MODEL_PATH = './model/'
saver = tf.train.Saver()

# 모델을 쉽게 재사용 할 수 있도록
# 텐서플로 컬렉션(collection)에 저장
train_vars = {'inputs': inputs, 'labels': labels,
              'hidden1': hidden1, 'hidden2': hidden2,
              'hidden3': hidden3, 'hidden4': hidden4,
              'hidden5': hidden5, 'logits': logits}

for key, var in train_vars.items():
    tf.add_to_collection(key, var)

# Train
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(train_op, feed_dict={inputs: batch_x,
                                          labels: batch_y})

        # validation
        accuracy_val = accuracy.eval(feed_dict={inputs: valid_x, labels: valid_y})
        print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

    # model save
    save_path = saver.save(sess, os.path.join(MODEL_PATH, 'my_model.ckpt'))

epoch: 00, valid. Acc: 0.8768
epoch: 01, valid. Acc: 0.9276
epoch: 02, valid. Acc: 0.9462
epoch: 03, valid. Acc: 0.9544
epoch: 04, valid. Acc: 0.9570


### 학습된 모델을 이용해 4번째 레이어만 수정하기

이제 '학습된 모델 저장하기' 에서 저장한 `'my_model.ckpt'`을 이용해, 4번째 hidden layer의 노드 수를 20개로 수정한 뒤 새로운 모델을 학습시키는 코드입니다. 아래의 코드는 위의 코드에서 `tf.add_to_collection`에 저장한 `inputs, labels, hidden3 `를 불러온 뒤, `new_hidden4, new_logits`을 추가한 새로운 모델을 학습하여 `my_new_model.ckpt`에 저장하는 코드입니다.

In [ ]:
reset_graph()

#################
# layers params #
#################
n_hidden4 = 20  # new hidden
n_outputs = 10  # new output

MODEL_PATH = './model/'
saver = tf.train.import_meta_graph(os.path.join(MODEL_PATH, 'my_model.ckpt.meta'))

inputs = tf.get_default_graph().get_collection('inputs')[0]
labels = tf.get_default_graph().get_collection('labels')[0]

hidden3 = tf.get_default_graph().get_collection('hidden3')[0]

new_hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name='new_hidden4')
new_logits = tf.layers.dense(new_hidden4, n_outputs, name='new_logits')

with tf.name_scope('new_loss'):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=new_logits))

################
# Hyper-params #
################
learning_rate = 0.001
n_epochs = 5
batch_size = 50

with tf.name_scope('new_train'):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('new_eval'):
    correct = tf.nn.in_top_k(new_logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# New Saver
new_saver = tf.train.Saver()

# Train the New Model
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, os.path.join(MODEL_PATH, 'my_model.ckpt'))

    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(train_op, feed_dict={inputs: batch_x,
                                          labels: batch_y})

        # validation
        accuracy_val = accuracy.eval(feed_dict={inputs: valid_x, labels: valid_y})
        print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

    # save the new model
    save_path = new_saver.save(sess, os.path.join(MODEL_PATH, 'my_new_model.ckpt'))

INFO:tensorflow:Restoring parameters from ./model/my_model.ckpt
epoch: 00, valid. Acc: 0.9548
epoch: 01, valid. Acc: 0.9732
epoch: 02, valid. Acc: 0.9696
epoch: 03, valid. Acc: 0.9746
epoch: 04, valid. Acc: 0.9752


# 텐서플로를 이용한 Transfer Learning

TensorFlow를 활용하여 Transfer Learning을 구현합니다. 사전에 학습된 모델 `my_model.ckpt`를 불러와 `hidden1과 hidden2` 레이어는 동결(Freezing)하여 그대로 사용하고, `hidden3` 레이어는 학습 가능한 상태로 유지합니다. 추가적으로, 새로운 `hidden4` 레이어와 `logits` 레이어를 정의하여 Transfer Learning을 수행합니다.

### 재사용할 레이어 동결(Freezing)하는 방법 (1)

먼저, 학습시킬 레이어(`hidden3, hidden4, logits`)와 동결(학습하지 않을)할 레이어(`hidden1, hidden2`)를 TensorFlow의 `tf.get_collection()`을 활용하여 설정해야 합니다.

- **학습시킬 레이어**(`hidden3, hidden4, logits`):  
  `tf.get_collection()`의 `scope` 인자에 정규표현식으로 학습 대상 레이어를 `'hidden[34]|logits'`와 같이 지정하면, 텐서의 `name`이 매칭되는 변수들을 찾을 수 있습니다. 이 변수들을 `optimizer.minimize()`의 `var_list` 인자에 전달하여 학습할 수 있습니다.

```python
# 학습시킬 레이어 설정 예시
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                               scope='hidden[34]|logits')  # 정규표현식
train_op = optimizer.minimize(loss, var_list=train_vars)
```

- **재사용할 레이어**(`hidden1~3`):  
  이와 유사하게, `tf.get_collection()`의 `scope` 인자에 정규표현식으로 `'hidden[123]'`을 지정하여 필요한 변수들을 찾습니다. 이후, 해당 변수들을 `tf.train.Saver()`에 전달하여 저장된 모델에서 복원합니다.

```python
# 재사용할 레이어 불러오기 예시
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope='hidden[123]')  # 정규표현식
restore_saver = tf.train.Saver(reuse_vars)

with tf.Session() as sess:
    restore_saver.restore(sess, './model/my_model.ckpt')
```

위 내용을 기반으로, 하위층(low layer)을 동결한 상태에서 새로운 레이어를 추가하여 모델을 학습할 수 있습니다.

In [ ]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300  # Reusing
n_hidden2 = 50  # Reusing
n_hidden3 = 50  # Reusing
n_hidden4 = 20  # New
n_outputs = 10  # New

inputs = tf.placeholder(tf.float32, shape=[None, n_inputs], name='inputs')
labels = tf.placeholder(tf.int32, shape=[None], name='labels')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(inputs, n_hidden1,
                              activation=tf.nn.relu, name='hidden1')  # Reusing
    hidden2 = tf.layers.dense(hidden1, n_hidden2,
                              activation=tf.nn.relu, name='hidden2')  # Reusing
    hidden3 = tf.layers.dense(hidden2, n_hidden3,
                              activation=tf.nn.relu, name='hidden3')  # Reusing
    hidden4 = tf.layers.dense(hidden3, n_hidden4,
                              activation=tf.nn.relu, name='hidden4')  # New
    logits = tf.layers.dense(hidden4, n_outputs, name='logits')  # new

with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

################
# Hyper-params #
################
learning_rate = 0.01
n_epochs = 5
batch_size = 50

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                   scope='hidden[34]|logits')
    train_op = optimizer.minimize(cross_entropy, var_list=train_vars)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# New Saver 정의
MODEL_PATH = './model/'
new_saver = tf.train.Saver()

# Reusing layer load
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope='hidden[123]')
restore_saver = tf.train.Saver(reuse_vars)

# Train the New Model
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    restore_saver.restore(sess, os.path.join(MODEL_PATH, 'my_model.ckpt'))

    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(train_op, feed_dict={inputs: batch_x,
                                          labels: batch_y})

        # validation
        accuracy_val = accuracy.eval(feed_dict={inputs: valid_x, labels: valid_y})
        print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

    # save the new model
    save_path = new_saver.save(sess, os.path.join(MODEL_PATH, 'my_transfer_model.ckpt'))

INFO:tensorflow:Restoring parameters from ./model/my_model.ckpt
epoch: 00, valid. Acc: 0.9480
epoch: 01, valid. Acc: 0.9516
epoch: 02, valid. Acc: 0.9580
epoch: 03, valid. Acc: 0.9578
epoch: 04, valid. Acc: 0.9584


### 재사용할 레이어 동결(Freezing)하는 방법 (2)

위에서 설명한 `optimizer.minimize()`의 `var_list` 인자를 활용하여 학습할 레이어를 지정하는 방식 대신, `tf.stop_gradient()`를 사용하여 Transfer Learning을 수행할 수도 있습니다. 아래 예시와 같이, 동결(freezing)하려는 마지막 레이어(예: `hidden2`) 바로 뒤에 `tf.stop_gradient()`를 적용하면 해당 레이어에서의 그래디언트 전파가 멈추게 됩니다.

```python
# tf.stop_gradient()를 사용한 Transfer Learning
hidden2 = tf.layers.dense(hidden1, ...)
hidden2_stop = tf.stop_gradient(hidden2)
hidden3 = tf.layers.dense(hidden2_stop, ...)
# ...
```

아래는 위에서 설명한 방식을 `tf.stop_gradient()`를 활용하여 작성한 코드입니다:

In [ ]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300  # Reusing
n_hidden2 = 50  # Reusing
n_hidden3 = 50  # Reusing
n_hidden4 = 20  # New
n_outputs = 10  # New

inputs = tf.placeholder(tf.float32, shape=[None, n_inputs], name='inputs')
labels = tf.placeholder(tf.int32, shape=[None], name='labels')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(inputs, n_hidden1,
                              activation=tf.nn.relu, name='hidden1')  # Reusing
    hidden2 = tf.layers.dense(hidden1, n_hidden2,
                              activation=tf.nn.relu, name='hidden2')  # Reusing
    hidden2_stop = tf.stop_gradient(hidden2)  # freezing
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3,
                              activation=tf.nn.relu, name='hidden3')  # Reusing
    hidden4 = tf.layers.dense(hidden3, n_hidden4,
                              activation=tf.nn.relu, name='hidden4')  # New
    logits = tf.layers.dense(hidden4, n_outputs, name='logits')  # new

with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

################
# Hyper-params #
################
learning_rate = 0.01
n_epochs = 5
batch_size = 50

with tf.name_scope('train'):
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# New Saver 정의
MODEL_PATH = './model/'
new_saver = tf.train.Saver()

# Reusing layer load
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope='hidden[123]')
restore_saver = tf.train.Saver(reuse_vars)

# Train the New Model
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    restore_saver.restore(sess, os.path.join(MODEL_PATH, 'my_model.ckpt'))

    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(train_op, feed_dict={inputs: batch_x,
                                          labels: batch_y})

        # validation
        accuracy_val = accuracy.eval(feed_dict={inputs: valid_x, labels: valid_y})
        print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

    # save the new model
    save_path = new_saver.save(sess, os.path.join(MODEL_PATH, 'my_transfer_model2.ckpt'))

INFO:tensorflow:Restoring parameters from ./model/my_model.ckpt
epoch: 00, valid. Acc: 0.9504
epoch: 01, valid. Acc: 0.9544
epoch: 02, valid. Acc: 0.9554
epoch: 03, valid. Acc: 0.9562
epoch: 04, valid. Acc: 0.9576


### 동결된 층 캐싱하기

위에서 `hidden1`과 `hidden2` 레이어를 재사용하면서 동결(Freezing)했는데, 이처럼 동결된 레이어는 학습 과정에서 변하지 않으므로, 가장 마지막 동결된 레이어(`hidden2`)의 출력을 캐싱(Caching)하여 활용할 수 있습니다. 이는 학습 속도를 개선하는 데 유용합니다.

1. **전체 Training Set 처리:**  
   Training Set 전체를 한 번 실행하여, 마지막 동결된 레이어(`hidden2`)의 출력을 얻습니다. (충분한 메모리가 있다고 가정합니다.)

2. **미니배치 구성:**  
   학습하는 동안, Training Set의 원본 데이터 대신 1단계에서 캐싱한 `hidden2` 레이어의 출력으로 미니배치를 구성하여, 다음 레이어에 입력으로 사용합니다.

In [ ]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300  # Reusing
n_hidden2 = 50  # Reusing
n_hidden3 = 50  # Reusing
n_hidden4 = 20  # New
n_outputs = 10  # New

inputs = tf.placeholder(tf.float32, shape=[None, n_inputs], name='inputs')
labels = tf.placeholder(tf.int32, shape=[None], name='labels')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(inputs, n_hidden1,
                              activation=tf.nn.relu, name='hidden1')  # Reusing
    hidden2 = tf.layers.dense(hidden1, n_hidden2,
                              activation=tf.nn.relu, name='hidden2')  # Reusing
    hidden2_stop = tf.stop_gradient(hidden2)  # freezing
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3,
                              activation=tf.nn.relu, name='hidden3')  # Reusing
    hidden4 = tf.layers.dense(hidden3, n_hidden4,
                              activation=tf.nn.relu, name='hidden4')  # New
    logits = tf.layers.dense(hidden4, n_outputs, name='logits')  # new

with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))

################
# Hyper-params #
################
learning_rate = 0.01
n_epochs = 5
batch_size = 50

with tf.name_scope('train'):
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# New Saver 정의
MODEL_PATH = './model/'
new_saver = tf.train.Saver()

# Reusing layer load
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope='hidden[123]')
restore_saver = tf.train.Saver(reuse_vars)

# Train
n_batches = len(train_x) // batch_size

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    restore_saver.restore(sess, os.path.join(MODEL_PATH, 'my_model.ckpt'))

    # Caching
    h2_cache = sess.run(hidden2, feed_dict={inputs: train_x})
    h2_cache_valid = sess.run(hidden2, feed_dict={inputs: valid_x})

    for epoch in range(n_epochs):
        # mini-batch for hidden2
        shuffle_idx = np.random.permutation(len(train_x))
        hidden2_batches = np.array_split(h2_cache[shuffle_idx], n_batches)
        label_batches = np.array_split(train_y[shuffle_idx], n_batches)
        for hidden2_batch, label_batch in zip(hidden2_batches, label_batches):
            sess.run(train_op, feed_dict={hidden2: hidden2_batch,
                                          labels: label_batch})

        accuracy_val = accuracy.eval(feed_dict={hidden2: h2_cache_valid,
                                                labels: valid_y})
        print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

    # save the new model
    save_path = new_saver.save(sess, os.path.join(MODEL_PATH, 'my_caching_model.ckpt'))

INFO:tensorflow:Restoring parameters from ./model/my_model.ckpt
epoch: 00, valid. Acc: 0.9504
epoch: 01, valid. Acc: 0.9544
epoch: 02, valid. Acc: 0.9554
epoch: 03, valid. Acc: 0.9562
epoch: 04, valid. Acc: 0.9576


# Task1:
#### 1. **기존 모델 생성 및 저장**
아래 코드를 실행하여 `my_model.h5` 파일을 생성하세요. 이 파일은 학습된 기본 모델로, Transfer Learning 과제에서 활용됩니다.


#### 2. **Transfer Learning 과제**
이제 학습된 모델(`my_model.h5`)을 기반으로 Transfer Learning을 수행합니다.

#### 3. **과제 목표**
1. `my_model.h5`에서 `hidden1`과 `hidden2` 레이어를 **동결(Freezing)** 처리합니다.
2. 새로운 **hidden4**와 **logits** 레이어를 추가하여 모델을 확장합니다.
3. 확장된 모델을 사용하여 **MNIST 데이터셋**을 학습하고, 테스트 데이터로 **성능을 평가**합니다.

#### 4. **과제 지시사항**
- `my_model.h5` 파일을 불러와 `hidden1`, `hidden2`, `hidden3`를 재사용하고, 새로운 `hidden4`와 `logits`를 추가하세요.
- `hidden1`과 `hidden2`는 동결 처리하여 학습되지 않도록 설정합니다.
- Transfer Learning 모델을 **Adam 옵티마이저**, **Sparse Categorical Crossentropy** 손실 함수, **Accuracy** 메트릭으로 컴파일하세요.
- 모델을 학습한 후 **테스트 정확도**를 출력하세요.




In [ ]:
# 기존 모델 생성 및 저장
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 기존 모델 생성
def create_base_model():
    model = Sequential([
        Dense(128, activation='relu', input_shape=(784,), name='hidden1'),
        Dense(64, activation='relu', name='hidden2'),
        Dense(32, activation='relu', name='hidden3'),
        Dense(10, activation='softmax', name='output'),
    ])
    return model

# MNIST 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

# 모델 생성 및 컴파일
base_model = create_base_model()
base_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# 모델 학습
base_model.fit(x_train, y_train, batch_size=64, epochs=5)

# 모델 저장
base_model.save('./my_model.h5')  # 'my_model.ckpt'는 Keras에서 'h5' 형식으로 저장됩니다.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Input

# TODO: 저장된 기존 모델('my_model.h5')을 불러오세요.
base_model = __________________________

# TODO: 새로운 입력 레이어를 정의하세요. (입력 형태: (784,))
input_layer = __________________________

# TODO: 기존 모델에서 hidden1부터 hidden3까지를 가져와 연결하세요.
hidden1 = __________________________
hidden2 = __________________________
hidden3 = __________________________

# TODO: hidden1과 hidden2 레이어를 동결하세요.
for layer in __________________________:
    __________________________

# TODO: 새로운 hidden4와 logits 레이어를 추가하여 모델을 확장하세요.
hidden4 = __________________________
logits = __________________________

# TODO: 새로운 Transfer Learning 모델을 생성하세요.
transfer_model = __________________________

# TODO: 모델을 컴파일하세요. (옵티마이저: Adam, 손실 함수: Sparse Categorical Crossentropy, 메트릭: Accuracy)
transfer_model.compile(
    optimizer=__________________________,
    loss=__________________________,
    metrics=__________________________
)

# TODO: MNIST 데이터셋을 로드하고 전처리하세요. (데이터를 (784,)로 변환 및 정규화)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = __________________________
x_test = __________________________

# TODO: 모델을 학습하세요. (배치 크기: 64, 에포크: 5, 검증 데이터: 20%)
history = transfer_model.fit(
    __________________________,
    batch_size=________________________,
    epochs=________________________,
    validation_split=________________________
)

# TODO: 모델을 평가하고 테스트 정확도를 출력하세요.
test_loss, test_acc = __________________________
print(f"테스트 정확도: {test_acc:.4f}")
